In [26]:
# Load Libraries
import pandas as pd
import numpy as np
import re
import emoji
from collections import Counter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [27]:
# Load Data
df = pd.read_csv('/Users/zhanghanshi/Desktop/WM_Project/my_project/nvda_twits_combined_sorted.csv')
df.head()

,Unnamed: 0,id,body,created_at,user,source,symbols,mentioned_users,entities,likes,links,conversation,reshare_message,reshares,owned_symbols,structurable
0,14999,12994205,$NVDA shorts using a lot of ammo trying to kee...,2013-04-11 19:34:19+00:00,"{'id': 18393, 'username': 'fatcoin888', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14998,12995313,"$NVDA quite a rebound today, i can hardly beli...",2013-04-11 20:03:53+00:00,"{'id': 37084, 'username': 'peter5slo', 'name':...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14997,12995587,As I said just over a week ago that $NVDA woul...,2013-04-11 20:12:52+00:00,"{'id': 27639, 'username': 'CDMCapital', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': {'basic': 'Bullish'}},NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14996,12995655,$NVDA shorting here is like picking up dimes i...,2013-04-11 20:15:08+00:00,"{'id': 27639, 'username': 'CDMCapital', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': {'basic': 'Bullish'}},"{'total': 1, 'user_ids': [55546]}",NaN,NaN,NaN,NaN,NaN,NaN
4,14995,12997231,Nvidia investing in â€˜once in a lifetime oppo...,2013-04-11 21:35:07+00:00,"{'id': 222797, 'username': 'StockTwitsLinks', ...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,"[{'title': ""Nvidia investing in 'once in a lif...",NaN,NaN,NaN,NaN,NaN


In [28]:
# Check original shape
print("Original shape:", df.shape)

# Calculate and print missing value count and percentage
missing_stats = df.isna().sum().to_frame(name='missing_count')
missing_stats['missing_percent'] = (missing_stats['missing_count'] / len(df)) * 100

# Print missing statistics
print("\nMissing value statistics:")
print(missing_stats)

# Define threshold and drop columns with more than 95% missing values
threshold = 85  # percent
cols_to_drop = missing_stats[missing_stats['missing_percent'] > threshold].index.tolist()
print(f"\nColumns to drop (>{threshold}% missing): {cols_to_drop}")

df_cleaned = df.drop(columns=cols_to_drop)

df_cleaned.head()

Original shape: (584181, 16)

Missing value statistics:
                 missing_count  missing_percent
Unnamed: 0                   0         0.000000
id                           0         0.000000
body                         0         0.000000
created_at                   0         0.000000
user                         0         0.000000
source                       0         0.000000
symbols                      0         0.000000
mentioned_users              0         0.000000
entities                     0         0.000000
likes                   270590        46.319548
links                   505507        86.532599
conversation            416995        71.381130
reshare_message         560185        95.892369
reshares                561183        96.063206
owned_symbols           566319        96.942386
structurable            583984        99.966278

Columns to drop (>85% missing): ['links', 'reshare_message', 'reshares', 'owned_symbols', 'structurable']


,Unnamed: 0,id,body,created_at,user,source,symbols,mentioned_users,entities,likes,conversation
0,14999,12994205,$NVDA shorts using a lot of ammo trying to kee...,2013-04-11 19:34:19+00:00,"{'id': 18393, 'username': 'fatcoin888', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,NaN
1,14998,12995313,"$NVDA quite a rebound today, i can hardly beli...",2013-04-11 20:03:53+00:00,"{'id': 37084, 'username': 'peter5slo', 'name':...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,NaN
2,14997,12995587,As I said just over a week ago that $NVDA woul...,2013-04-11 20:12:52+00:00,"{'id': 27639, 'username': 'CDMCapital', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': {'basic': 'Bullish'}},NaN,NaN
3,14996,12995655,$NVDA shorting here is like picking up dimes i...,2013-04-11 20:15:08+00:00,"{'id': 27639, 'username': 'CDMCapital', 'name'...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': {'basic': 'Bullish'}},"{'total': 1, 'user_ids': [55546]}",NaN
4,14995,12997231,Nvidia investing in â€˜once in a lifetime oppo...,2013-04-11 21:35:07+00:00,"{'id': 222797, 'username': 'StockTwitsLinks', ...","{'id': 1, 'title': 'Stocktwits', 'url': 'https...","[{'id': 2925, 'symbol': 'NVDA', 'title': 'NVID...",[],{'sentiment': None},NaN,NaN


In [30]:
import pandas as pd
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Initialize stemmer and stopword set
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Define advertising-related keywords
advertising_keywords = [
    "learn trading", "get alerts", "get ideas", "get updates", "get analysis",
    "welcome to discord", "welcome to the discord group", "bitcoin", "crypto",
    "top analyst price target", "top analyst target price", "top analyst target for next week"
]

def is_advertising(text: str) -> bool:
    """Filter out tweets with links or promotional keywords"""
    if not isinstance(text, str):
        return False
    txt = text.lower()
    if 'https' in txt or 'http' in txt or 'www' in txt:
        return True
    return any(keyword in txt for keyword in advertising_keywords)

def clean_tweet(text: str) -> str:
    """Clean and normalize tweet text"""
    if pd.isna(text):
        return ""

    text = str(text).strip().lower()

    # Convert emojis to text format
    text = emoji.demojize(text)
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    
    # Remove @mention but keep the name
    text = re.sub(r"@(\w+)", r"\1", text)
    
    # Remove "#" but keep hashtag word
    text = re.sub(r"#(\w+)", r"\1", text)
    
    # Keep alphanumerics, punctuation, emoji text, $, and basic signs
    text = re.sub(r"[^\w\s$.,!?':]", '', text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

def clean_and_stem(text: str) -> str:
    """Apply stemming and remove stopwords"""
    tokens = word_tokenize(text)
    words = [w for w in tokens if w.isalpha() and w not in stop_words]  # 只保留字母词
    stemmed = [stemmer.stem(w) for w in words]
    return " ".join(stemmed)

def extract_emoji_texts(text: str):
    """Extract valid emoji names using emoji.demojize and a clean match"""
    text = emoji.demojize(str(text))  # ensure demojized form
    candidates = re.findall(r':[a-zA-Z0-9_]+:', text)
    # Keep only those that actually map to some emoji when re-emojized
    return [c for c in candidates if emoji.emojize(c, language='alias') != c]

# Filter advertising tweets
before = len(df_cleaned)
df_cleaned = df_cleaned[~df_cleaned["body"].apply(is_advertising)].copy()
after = len(df_cleaned)
print(f"Total: {before}, After ad filtering: {after}, Removed: {before - after}")

# Clean tweet body
df_cleaned["Cleaned_Tweet"] = df_cleaned["body"].apply(clean_tweet)

# Remove empty cleaned tweets
df_cleaned = df_cleaned[df_cleaned["Cleaned_Tweet"] != ""]

# Apply stemming and stopword removal
df_cleaned["Processed_Tweet"] = df_cleaned["Cleaned_Tweet"].apply(clean_and_stem)

# Extract emojis
df_cleaned["Emoji_Texts"] = df_cleaned["Cleaned_Tweet"].apply(extract_emoji_texts)

# Parse datetime and save date
df_cleaned["Date"] = pd.to_datetime(df_cleaned["created_at"], errors="coerce").dt.date



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhanghanshi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhanghanshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total: 501908, After ad filtering: 501908, Removed: 0


In [31]:
# Save to CSV
output_path = "/Users/zhanghanshi/Desktop/WM_Project/my_project/cleaned_nvda.csv"
df_cleaned.to_csv(output_path, index=False)
print(f"\nCleaned data saved to: {output_path}")


Cleaned data saved to: /Users/zhanghanshi/Desktop/WM_Project/my_project/cleaned_nvda.csv
